In [1]:
# Volume of the diff DEM labels is calculated

In [2]:
import numpy as np
import geopandas as gpd
import pandas as pd
import rasterio as rio
from rasterio.mask import mask
import os

In [3]:

def calculate_volume_change(row, diff_dem):
    try:
        # Create a mask of the DEM using the polygon
        out_image, out_transform = mask(diff_dem, [row.geometry], crop=True)

        polygon_diff_dem = np.where((out_image > 8000) | (out_image < -500), np.nan, out_image)

        # Calculate the total elevation change
        total_volume_change = np.nansum(polygon_diff_dem) * 100  # res_x (10 m) * res_y (10 m) * 1 m height change = 100 m^3

        return total_volume_change
    except ValueError:
        # Catch the exception when shapes do not overlap with the raster
        return np.nan

In [9]:
# Input parameters have to be set manually

#  'herschel': [2010],
# 'gydan': [2010],
# 'peel': [2010, 2011, 2013],
# 'kolguev': [2010]

# DEM Labels
region = 'kolguev'
year_start = '2010' # Different for Peel


file_path = f"labels\\labels_w_metrics\\{region}_DEM_labels_rasterized_SimpleMetrics.geojson"

object = gpd.read_file(file_path)

# DEM Directory
dem_directory = f"DEM_rasters\\DEM_raster_combined\\{region}\\DEM_merged_{region}_{year_start}_A_"

import os

# split the objects into different groups with the same year_end
groups = object.groupby('year_end')
merged_groups = []

for year_end, group in groups:
    year_end = int(year_end)
    print(year_end)
    
    # Check if DEM file exists for the current year
    dem_file_path = f"{dem_directory}{year_end}.tif"
    if not os.path.isfile(dem_file_path):
        print(f"DEM file not found for {year_end}. Skipping...")
        continue
    
    # Load the DEM
    with rio.open(dem_file_path) as dem:
        
        group['total_volume_change'] = group.apply(lambda row: calculate_volume_change(row, diff_dem=dem), axis=1)
    
    merged_groups.append(group)

# Merge all groups back into one GeoDataFrame
merged_object = gpd.GeoDataFrame(pd.concat(merged_groups, ignore_index=True), crs=object.crs)

# Save the merged GeoDataFrame to a single file
merged_object.to_file(f'Results/{region}/{region}_DEM_labels_results.geojson', driver='GeoJSON')
merged_object.to_file(f'Results/all/{region}_DEM_labels_results.geojson', driver='GeoJSON')

2017
